# HashiCorp Vault Demo for KMIP Secrets Engine and MongoDB Enterprise TDE

The OASIS Key Management Interoperability Protocol (KMIP) standard is a widely adopted protocol for handling cryptographic workloads and secrets management for enterprise infrastructure such as databases, network storage, and virtual/physical servers.  When an organization has services and applications that need to perform cryptographic operations (e.g.: transparent database encryption (TDE), full disk encryption, etc.), it often delegates the key management task to an external provider via KMIP protocol. 

This demo demonstrates HashiCorp Vault's KMIP Secrets Engine to allow Vault to function as a KMIP server for clients that that retrieve cryptographic keys for encrypting data via the KMIP protocol.

Note: MongoDB Community & Enterprise server editions have the same developer features (queries, aggregation, replication, sharding, etc).  MongoDB Enterprise Advanced adds:
- Operational & security features (In-memory storage engine, Auditing, Kerberos & LDAP authentication, Encryption at Rest). https://www.mongodb.com/docs/manual/administration/upgrade-community-to-enterprise/
- Entitlements to use additional tools including Ops Manager, BI Connector, and Enterprise Operator for Kubernetes.

Encryption at Rest, when used in conjunction with transport encryption and good security policies that protect relevant accounts, passwords, and encryption keys, can help ensure compliance with security and privacy standards, including HIPAA, PCI-DSS, and FERPA.  Note: Encryption at Rest is offered on Enterprise Edition only. 
- https://www.mongodb.com/docs/manual/core/security-encryption-at-rest/

<img src="images/vault-demo-kmip-mongodb-tde.png">

## Setup of the Demo

This setup is tested on MacOS and is meant to simulate a distributed setup.  The components used in this demo are:
- Vault Enterprise installed on docker (to simulate an external Vault)
- MongoDB Enterprise installed on docker (to simulate an external MongoDB)
- You have the Vault CLI installed

This assumes your Vault server is installed using docker and already running on http://127.0.0.1:8200
and you have set your VAULT_ADDR and VAULT_TOKEN variables.

Note: You will need Vault to be installed with an ADP KMIP license add-on.  Also note that a Premium license is required if you wish the KMIP listener to scale on the Vault cluster.  For Standard or Plus license, the KMIP listener is only on the leader node.

## Requirements to Run This Demo
You will need Visual Studio Code to be installed with the Jupyter plugin.  To run this notebook in VS Code, chose the Jupyter kernel and then Bash.
- To run the current cell, use Ctrl + Enter.
- To run the current cell and advance to the next, use Shift+Enter.

# Setup Pre-requisites (One-time)

Assumes you have docker installed and brew installed

- https://docs.docker.com/desktop/install/mac-install/
- https://brew.sh/

# Setting up HashiCorp Vault

In [ ]:
# Optional.  The following are some sample commands for running Vault Enterprise in docker.
# Expose both the Vault API and the KMIP ports to the host machine.
export VAULT_PORT=8200
export VAULT_KMIP_PORT=5696
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"
# Change the path to your license file
export VAULT_LICENSE=$(cat $HOME/vault-enterprise/vault_local/data/vault.hclic)
docker run -d --rm --name vault-enterprise --cap-add=IPC_LOCK \
-e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
-e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
-e "VAULT_LICENSE=${VAULT_LICENSE}" \
-p ${VAULT_KMIP_PORT}:${VAULT_KMIP_PORT} \
-p ${VAULT_PORT}:${VAULT_PORT} hashicorp/vault-enterprise:latest

# MongoDB Enterprise without Encryption at Rest

Setup a demo MongoDB server.  Show that data stored is unencrypted by default.

In [ ]:
# Use latest 7.0 MongoDB Enterprise docker image.  This supports both linux/amd64 and linux/arm64
# Ref: https://hub.docker.com/r/mongodb/mongodb-enterprise-server/tags
export MONGODB_TAG=7.0-ubuntu2204
export MONGODB_PORT=27017

# Start a MongoDB node 
docker run -d --rm -p $MONGODB_PORT:$MONGODB_PORT --name mongo-enterprise \
mongodb/mongodb-enterprise-server:$MONGODB_TAG --port $MONGODB_PORT --bind_ip "0.0.0.0"

In [ ]:
# Check that the Vault Server and MongoDB are running
docker ps

In [ ]:
# Show the contents of MongoDB database data folder
docker exec -it mongo-enterprise sh -c "ls /data/db"

In [ ]:
# set mongosh alias to make it easier to execute mongosh commands to the first MongoDB docker container
# Use the connection string for the replica set.  This will allow write operations to go to the primary node.
# As Authorization is turned on, we will be specifying the root admin user credentials in the connection string.
echo "MongoDB Port: $MONGODB_PORT"
alias mongosh="docker exec -it mongo-enterprise mongosh \"mongodb://127.0.0.1:$MONGODB_PORT/admin\""

In [ ]:
# Create database “record”, insert record with random user details (using function makeid), and display table records
# Note: You can run this multiple times to insert more records as required

# Take note of the username and password values being inserted into the table
mongosh --eval "use record" \
--eval "function makeid(length) {
    var result           = '';
    var characters       = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789';
    var charactersLength = characters.length;
    for ( var i = 0; i < length; i++ ) {
      result += characters.charAt(Math.floor(Math.random() * 
 charactersLength));
   }
   return result;
}" \
--eval "db.users.insertOne({username: \"user-\" + makeid(4), password: makeid(20)})" \
--eval "db.users.find()"

In [ ]:
# Now view the contents of MongoDB database data folder again.  You should notice a new collection file for the data we inserted. 
# It should be named like "collection-7--XXXXXXXXXXXXXXXXXX.wt" collection-7--4848515339651436823.wt 
docker exec -it mongo-enterprise sh -c "ls /data/db"

In [ ]:
# Dump the contents of the new collections file.
# Notice that the username and password of the inserted record can be seen in clear text
# Note: MongoDB might take a while to flush the data to the database file.  Wait a while and try again if you do not see any data in the file.
docker exec -it mongo-enterprise sh -c "cat data/db/collection-7*"

In [ ]:
# Stop MongoDB container.
docker stop mongo-enterprise

# MongoDB Enterprise with Encryption at Rest with Vault KMIP Engine

This section demonstrates the configuration of MongoDB Enterprise with Encryption at Rest with HashiCorp's Vault KMIP secrets engine.

Ref:
- https://developer.hashicorp.com/vault/docs/secrets/kmip
- https://developer.hashicorp.com/vault/tutorials/adp/kmip-engine

## Step 1 - Enable Vault KMIP Engine

In [ ]:
# Enable KMIP secrets engine at the default mount path
#vault secrets disable kmip
vault secrets enable kmip

In [ ]:
# Start KMIP Server and set client TLS certificate TTL to 365 days

# Get the docker IP address of the Vault server to configure the certificate SANs.  Otherwise the MongoDB KMIP connection will fail.
export VAULT_IP=$(docker inspect -f '{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' vault-enterprise)

# Option 1 - Elliptic Curve (EC) is the default
vault write kmip/config listen_addrs=0.0.0.0:$VAULT_KMIP_PORT default_tls_client_ttl=365d server_hostnames=$VAULT_IP,localhost
# Option 2 - RSA
#vault write kmip/config listen_addrs=0.0.0.0:$VAULT_KMIP_PORT tls_ca_key_type="rsa" tls_ca_key_bits=2048 default_tls_client_ttl=365d server_hostnames=$VAULT_IP,localhost

# Check the KMIP configuration
echo
vault read kmip/config

## Step 2 - Create KMIP Scope and Role in Vault

The KMIP secrets engine uses the concept of scopes to partition KMIP managed object storage into multiple named buckets. Within a scope, roles can be created which dictate the set of allowed operations that the particular role can perform. TLS client certificates can be generated for a role, which services and applications can then use when sending KMIP requests against Vault's KMIP secret engine.

In order to generate client certificates for KMIP clients to interact with Vault's KMIP server, we must first create a scope and role and specify the desired set of allowed operations for it.

In [ ]:
# Settings for the KMIP scope
export KMIP_SVC_NAME=mongodb-svc

# Create a scope
vault write -f kmip/scope/$KMIP_SVC_NAME

# List scopes and verify the KMIP scope has been created
echo
vault list kmip/scope

In [ ]:
# Settings for the KMIP role to be created
export KMIP_ROLE_NAME=tde

# Create the KMIP Role (note the TTL if not set, it uses the default Vault token/lease TTL of 768hrs)
vault write kmip/scope/$KMIP_SVC_NAME/role/$KMIP_ROLE_NAME tls_client_key_bits=2048 tls_client_key_type=rsa operation_all=true tls_client_ttl=365d

# List roles and verify the KMIP role has been created
echo
vault list kmip/scope/$KMIP_SVC_NAME/role

In [ ]:
# Optional - View the role details
vault read kmip/scope/$KMIP_SVC_NAME/role/$KMIP_ROLE_NAME

## Step 3 - Create the KMIP certificates for MongoDB Encryption at Rest

In [ ]:
# Specify certificate folder path
export CERT_PATH=certs

# Create a folder for the KMIP certificates
mkdir $CERT_PATH

# Save KMIP CA certificate
vault read -format=json kmip/ca | jq -r .data.ca_pem > $CERT_PATH/ca.pem    

# Generate the required certificates
vault write -format=json \
  kmip/scope/$KMIP_SVC_NAME/role/$KMIP_ROLE_NAME/credential/generate \
  format=pem > credential.json

# Save the certificate and private key
jq -r .data.certificate < credential.json > $CERT_PATH/cert.pem
jq -r .data.private_key < credential.json > $CERT_PATH/key.pem
cat $CERT_PATH/cert.pem $CERT_PATH/key.pem > $CERT_PATH/client.pem

# Remove the temp file
rm credential.json

## Step 4 - Configuring MongoDB Enterprise Server with Encryption at Rest

In [ ]:
echo "Vault IP Address is: $VAULT_IP"
echo "Vault KMIP Port is: $VAULT_KMIP_PORT"

# Start a MongoDB node 
docker run -d --rm --volume=./certs:/certs -p $MONGODB_PORT:$MONGODB_PORT --name mongo-enterprise \
mongodb/mongodb-enterprise-server:$MONGODB_TAG --port $MONGODB_PORT --bind_ip "0.0.0.0" \
--enableEncryption --kmipServerName $VAULT_IP --kmipPort $VAULT_KMIP_PORT \
--kmipServerCAFile /certs/ca.pem --kmipClientCertificateFile /certs/client.pem

In [ ]:
# Check that the Vault Server and MongoDB are running
docker ps

## Step 5 - Testing MongoDB Enterprise Encryption at Rest

In [ ]:
# Show the contents of MongoDB database data folder
docker exec -it mongo-enterprise sh -c "ls /data/db"

In [ ]:
# Create database “record”, insert record with random user details (using function makeid), and display table records
# Note: You can run this multiple times to insert more records as required

# Take note of the username and password values being inserted into the table
mongosh --eval "use record" \
--eval "function makeid(length) {
    var result           = '';
    var characters       = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789';
    var charactersLength = characters.length;
    for ( var i = 0; i < length; i++ ) {
      result += characters.charAt(Math.floor(Math.random() * 
 charactersLength));
   }
   return result;
}" \
--eval "db.users.insertOne({username: \"user-\" + makeid(4), password: makeid(20)})" \
--eval "db.users.find()"

In [ ]:
# Simulate that the KMIP server is down by changing the listening port to 5697
vault write kmip/config listen_addrs=0.0.0.0:5697

In [ ]:
# Reinsert another record and query to show that CRUD operations are still working
mongosh --eval "use record" \
--eval "function makeid(length) {
    var result           = '';
    var characters       = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789';
    var charactersLength = characters.length;
    for ( var i = 0; i < length; i++ ) {
      result += characters.charAt(Math.floor(Math.random() * 
 charactersLength));
   }
   return result;
}" \
--eval "db.users.insertOne({username: \"user-\" + makeid(4), password: makeid(20)})" \
--eval "db.users.find()"

In [ ]:
# Now view the contents of MongoDB database data folder again.  You should notice a new collection file for the data we inserted. 
# It should be named like "collection-7--XXXXXXXXXXXXXXXXXX.wt" collection-7--4848515339651436823.wt 
docker exec -it mongo-enterprise sh -c "ls /data/db"

In [ ]:
# Dump the contents of the new collections file.
# Notice that the username and password of the inserted record is now encrypted
# Note: MongoDB might take a while to flush the data to the database file.  Wait a while and try again if you do not see any data in the file.
docker exec -it mongo-enterprise sh -c "cat data/db/collection-7*"

# Cleanup

In [ ]:
# Cleanup

# Disable KMIP secrets engine
vault secrets disable kmip

# Stop Vault container
docker stop vault-enterprise

# Stop MongoDB container
docker stop mongo-enterprise

# Remove KMIP certificate demo files
rm -rf $CERT_PATH